In [22]:
#loading packages and plot figure layout
import pandas as pd
import pandas_datareader as pdr
from IPython.display import display
import numpy as np
import random
import datetime
import matplotlib.pyplot as plt
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
plt.rcParams.update({'font.size': 14})
import datetime as dt
end_date = datetime.datetime.now().strftime("%Y-%m-%d")

import yfinance as yf

# Define the ticker symbol
tickerSymbol = 'AAPL'

# Get data on the ticker
tickerData = yf.Ticker(tickerSymbol)

# Get the historical prices for this ticker
tickerDf = tickerData.history(period='1d', start='2022-01-01', end='2022-04-20')

# Print the data
print(tickerDf)


#importing the venn diagram package, if a venn diagram is needed
#%pip install matplotlib-venn
from matplotlib_venn import venn2
import matplotlib.pyplot as plt
import plotly.graph_objects as go


                                 Open        High         Low       Close   
Date                                                                        
2022-01-03 00:00:00-05:00  176.534331  181.547540  176.415210  180.683868  \
2022-01-04 00:00:00-05:00  181.299361  181.607100  177.814925  178.390701   
2022-01-05 00:00:00-05:00  178.301354  178.857271  173.367564  173.645523   
2022-01-06 00:00:00-05:00  171.441708  174.022770  170.389433  170.746811   
2022-01-07 00:00:00-05:00  171.630298  172.871190  169.783849  170.915543   
...                               ...         ...         ...         ...   
2022-04-12 00:00:00-04:00  167.008300  168.847151  165.636604  166.650467   
2022-04-13 00:00:00-04:00  166.382092  170.010108  165.765830  169.373962   
2022-04-14 00:00:00-04:00  169.592663  170.238758  164.046259  164.294754   
2022-04-18 00:00:00-04:00  162.933011  165.596882  162.585127  164.076096   
2022-04-19 00:00:00-04:00  164.026385  166.809529  162.923068  166.392044   

In [26]:
# Set start and end dates
start = dt.datetime(2010, 4, 1)
end = dt.datetime.now()

# Define tickers and data
tickers = ['KO', 'PEP']
tickerDf_coca = yf.download('KO', start=start, end=end)
tickerDf_pepsi = yf.download('PEP', start=start, end=end)

# Calculate yearly percentage growth
price_change_coca = tickerDf_coca['Close'].pct_change()
price_change_pepsi = tickerDf_pepsi['Close'].pct_change()
avg_yearly_price_change_coca = ((1 + price_change_coca).resample('Y').prod() - 1).mean() * 100
avg_yearly_price_change_pepsi = ((1 + price_change_pepsi).resample('Y').prod() - 1).mean() * 100

# Create a table of the yearly percentage growth for each ticker
growth_table = pd.DataFrame({'Coca-Cola': avg_yearly_price_change_coca, 'PepsiCo': avg_yearly_price_change_pepsi}, index=[start.year, end.year])
growth_table.columns = ['Average Yearly Percentage Growth for ' + col + ' Stock' for col in growth_table.columns]

# Print the growth table
print("\nYearly Percentage Growth for Coca-Cola and PepsiCo Stocks")
print(growth_table)

# Print a table of the stock prices over the last year
last_year_coca = tickerDf_coca.tail(252)['Close']
last_year_pepsi = tickerDf_pepsi.tail(252)['Close']
last_year_table = pd.DataFrame({'Coca-Cola': last_year_coca, 'PepsiCo': last_year_pepsi})
last_year_table.columns = ['Last Year\'s Closing Price for ' + col + ' Stock' for col in last_year_table.columns]

# Print the last year's price table
print("\nLast Year's Closing Stock Prices for Coca-Cola and PepsiCo Stocks")
print(last_year_table)

# Create a table of the percentage growth each year
yearly_growth_coca = price_change_coca.resample('Y').apply(lambda x: (1+x).prod() - 1) * 100
yearly_growth_pepsi = price_change_pepsi.resample('Y').apply(lambda x: (1+x).prod() - 1) * 100
yearly_growth_table = pd.concat([yearly_growth_coca, yearly_growth_pepsi], axis=1)
yearly_growth_table.columns = ['Coca-Cola', 'PepsiCo']
yearly_growth_table.columns = ['Yearly Percentage Growth for ' + col + ' Stock' for col in yearly_growth_table.columns]

# Print the yearly growth table
print("\nYearly Percentage Growth for Coca-Cola and PepsiCo Stocks")
print(yearly_growth_table.round(2))


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Yearly Percentage Growth for Coca-Cola and PepsiCo Stocks
      Average Yearly Percentage Growth for Coca-Cola Stock   
2010                                           6.336157     \
2023                                           6.336157      

      Average Yearly Percentage Growth for PepsiCo Stock  
2010                                           7.900508   
2023                                           7.900508   

Last Year's Closing Stock Prices for Coca-Cola and PepsiCo Stocks
            Last Year's Closing Price for Coca-Cola Stock   
Date                                                        
2022-04-19                                      65.070000  \
2022-04-20                                      65.959999   
2022-04-21                                      66.209999   
2022-04-22                                      65.250000   
2022-0

In [30]:
# Set start and end dates
start = dt.datetime(2010, 4, 1)
end = dt.datetime.now()

# Define tickers and data
tickers = ['NEE', 'XOM']
tickerDf_nee = yf.download('NEE', start=start, end=end)
tickerDf_xom = yf.download('XOM', start=start, end=end)

# Calculate yearly percentage growth
price_change_nee = tickerDf_nee['Close'].pct_change()
price_change_xom = tickerDf_xom['Close'].pct_change()
avg_yearly_price_change_nee = ((1 + price_change_nee).resample('Y').prod() - 1).mean() * 100
avg_yearly_price_change_xom = ((1 + price_change_xom).resample('Y').prod() - 1).mean() * 100

# Create a table of the yearly percentage growth for each ticker
growth_table = pd.DataFrame({'NextEra Energy': avg_yearly_price_change_nee, 'ExxonMobil': avg_yearly_price_change_xom}, index=[start.year, end.year])
growth_table.columns = ['Average Yearly Percentage Growth for ' + col + ' Stock' for col in growth_table.columns]

# Print the growth table
print("\nYearly Percentage Growth for NextEra Energy and ExxonMobil Stocks")
print(growth_table)

# Print a table of the stock prices over the last year
last_year_nee = tickerDf_nee.tail(252)['Close']
last_year_xom = tickerDf_xom.tail(252)['Close']
last_year_table = pd.DataFrame({'NextEra Energy': last_year_nee, 'ExxonMobil': last_year_xom})
last_year_table.columns = ['Last Year\'s Closing Price for ' + col + ' Stock' for col in last_year_table.columns]

# Print the last year's price table
print("\nLast Year's Closing Stock Prices for NextEra Energy and ExxonMobil Stocks")
print(last_year_table)

# Create a table of the percentage growth each year
yearly_growth_nee = price_change_nee.resample('Y').apply(lambda x: (1+x).prod() - 1) * 100
yearly_growth_xom = price_change_xom.resample('Y').apply(lambda x: (1+x).prod() - 1) * 100
yearly_growth_table = pd.concat([yearly_growth_nee, yearly_growth_xom], axis=1)
yearly_growth_table.columns = ['NextEra Energy', 'ExxonMobil']
yearly_growth_table.columns = ['Yearly Percentage Growth for ' + col + ' Stock' for col in yearly_growth_table.columns]

# Print the yearly growth table
print("\nYearly Percentage Growth for NextEra Energy and ExxonMobil Stocks")
print(yearly_growth_table.round(2))


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Yearly Percentage Growth for NextEra Energy and ExxonMobil Stocks
      Average Yearly Percentage Growth for NextEra Energy Stock   
2010                                          15.113253          \
2023                                          15.113253           

      Average Yearly Percentage Growth for ExxonMobil Stock  
2010                                           7.468267      
2023                                           7.468267      

Last Year's Closing Stock Prices for NextEra Energy and ExxonMobil Stocks
            Last Year's Closing Price for NextEra Energy Stock   
Date                                                             
2022-04-19                                          81.830002   \
2022-04-20                                          81.510002    
2022-04-21                                          76.180000    
20

In [31]:
# Set start and end dates
start = dt.datetime(2010, 4, 1)
end = dt.datetime.now()

# Define tickers and data
tickers = ['TSLA', 'F']
tickerDf_tesla = yf.download('TSLA', start=start, end=end)
tickerDf_ford = yf.download('F', start=start, end=end)

# Calculate yearly percentage growth
price_change_tesla = tickerDf_tesla['Close'].pct_change()
price_change_ford = tickerDf_ford['Close'].pct_change()
avg_yearly_price_change_tesla = ((1 + price_change_tesla).resample('Y').prod() - 1).mean() * 100
avg_yearly_price_change_ford = ((1 + price_change_ford).resample('Y').prod() - 1).mean() * 100

# Create a table of the yearly percentage growth for each ticker
growth_table = pd.DataFrame({'Tesla': avg_yearly_price_change_tesla, 'Ford': avg_yearly_price_change_ford}, index=[start.year, end.year])
growth_table.columns = ['Average Yearly Percentage Growth for ' + col + ' Stock' for col in growth_table.columns]

# Print the growth table
print("\nYearly Percentage Growth for Tesla and Ford Stocks")
print(growth_table)

# Print a table of the stock prices over the last year
last_year_tesla = tickerDf_tesla.tail(252)['Close']
last_year_ford = tickerDf_ford.tail(252)['Close']
last_year_table = pd.DataFrame({'Tesla': last_year_tesla, 'Ford': last_year_ford})
last_year_table.columns = ['Last Year\'s Closing Price for ' + col + ' Stock' for col in last_year_table.columns]

# Print the last year's price table
print("\nLast Year's Closing Stock Prices for Tesla and Ford Stocks")
print(last_year_table)

# Create a table of the percentage growth each year
yearly_growth_tesla = price_change_tesla.resample('Y').apply(lambda x: (1+x).prod() - 1) * 100
yearly_growth_ford = price_change_ford.resample('Y').apply(lambda x: (1+x).prod() - 1) * 100
yearly_growth_table = pd.concat([yearly_growth_tesla, yearly_growth_ford], axis=1)
yearly_growth_table.columns = ['Tesla', 'Ford']
yearly_growth_table.columns = ['Yearly Percentage Growth for ' + col + ' Stock' for col in yearly_growth_table.columns]

# Print the yearly growth table
print("\nYearly Percentage Growth for Tesla and Ford Stocks")
print(yearly_growth_table.round(2))


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Yearly Percentage Growth for Tesla and Ford Stocks
      Average Yearly Percentage Growth for Tesla Stock   
2010                                         91.378852  \
2023                                         91.378852   

      Average Yearly Percentage Growth for Ford Stock  
2010                                         6.545245  
2023                                         6.545245  

Last Year's Closing Stock Prices for Tesla and Ford Stocks
            Last Year's Closing Price for Tesla Stock   
Date                                                    
2022-04-19                                 342.716675  \
2022-04-20                                 325.733337   
2022-04-21                                 336.260010   
2022-04-22                                 335.016663   
2022-04-25                                 332.673340   
...     

In [29]:
# Set start and end dates
start = dt.datetime(2010, 4, 1)
end = dt.datetime.now()

# Define tickers and data
tickers = ['CVX', 'ORSTED.CO']
tickerDf_chevron = yf.download('CVX', start=start, end=end)
tickerDf_orsted = yf.download('ORSTED.CO', start=start, end=end)

# Calculate yearly percentage growth
price_change_chevron = tickerDf_chevron['Close'].pct_change()
price_change_orsted = tickerDf_orsted['Close'].pct_change()
avg_yearly_price_change_chevron = ((1 + price_change_chevron).resample('Y').prod() - 1).mean() * 100
avg_yearly_price_change_orsted = ((1 + price_change_orsted).resample('Y').prod() - 1).mean() * 100

# Create a table of the yearly percentage growth for each ticker
growth_table = pd.DataFrame({'Chevron': avg_yearly_price_change_chevron, 'Ørsted': avg_yearly_price_change_orsted}, index=[start.year, end.year])
growth_table.columns = ['Average Yearly Percentage Growth for ' + col + ' Stock' for col in growth_table.columns]

# Print the growth table
print("\nYearly Percentage Growth for Chevron and Ørsted Stocks")
print(growth_table)

# Print a table of the stock prices over the last year
last_year_chevron = tickerDf_chevron.tail(252)['Close']
last_year_orsted = tickerDf_orsted.tail(252)['Close']
last_year_table = pd.DataFrame({'Chevron': last_year_chevron, 'Ørsted': last_year_orsted})
last_year_table.columns = ['Last Year\'s Closing Price for ' + col + ' Stock' for col in last_year_table.columns]

# Print the last year's price table
print("\nLast Year's Closing Stock Prices for Chevron and Ørsted Stocks")
print(last_year_table)

# Create a table of the percentage growth each year
yearly_growth_chevron = price_change_chevron.resample('Y').apply(lambda x: (1+x).prod() - 1) * 100
yearly_growth_orsted = price_change_orsted.resample('Y').apply(lambda x: (1+x).prod() - 1) * 100
yearly_growth_table = pd.concat([yearly_growth_chevron, yearly_growth_orsted], axis=1)
yearly_growth_table.columns = ['Chevron', 'Ørsted']
yearly_growth_table.columns = ['Yearly Percentage Growth for ' + col + ' Stock' for col in yearly_growth_table.columns]

# Print the yearly growth table
print("\nYearly Percentage Growth for Chevron and Ørsted Stocks")
print(yearly_growth_table.round(2))


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Yearly Percentage Growth for Chevron and Ørsted Stocks
      Average Yearly Percentage Growth for Chevron Stock   
2022                                           2.195073   \
2023                                           2.195073    

      Average Yearly Percentage Growth for Ørsted Stock  
2022                                         -15.538777  
2023                                         -15.538777  

Last Year's Closing Stock Prices for Chevron and Ørsted Stocks
            Last Year's Closing Price for Chevron Stock   
Date                                                      
2022-04-19                                   171.830002  \
2022-04-20                                   172.529999   
2022-04-21                                   164.580002   
2022-04-22                                   160.949997   
2022-04-25                       

In [32]:
# Set start and end dates
start = dt.datetime(2010, 4, 1)
end = dt.datetime.now()

# Define tickers and data
tickers = ['JNJ', 'NVO']
tickerDf_jnj = yf.download('JNJ', start=start, end=end)
tickerDf_nvo = yf.download('NVO', start=start, end=end)

# Calculate yearly percentage growth
price_change_jnj = tickerDf_jnj['Close'].pct_change()
price_change_nvo = tickerDf_nvo['Close'].pct_change()
avg_yearly_price_change_jnj = ((1 + price_change_jnj).resample('Y').prod() - 1).mean() * 100
avg_yearly_price_change_nvo = ((1 + price_change_nvo).resample('Y').prod() - 1).mean() * 100

# Create a table of the yearly percentage growth for each ticker
growth_table = pd.DataFrame({'Johnson & Johnson': avg_yearly_price_change_jnj, 'Novo Nordisk A/S': avg_yearly_price_change_nvo}, index=[start.year, end.year])
growth_table.columns = ['Average Yearly Percentage Growth for ' + col + ' Stock' for col in growth_table.columns]

# Print the growth table
print("\nYearly Percentage Growth for Johnson & Johnson and Novo Nordisk A/S Stocks")
print(growth_table)

# Print a table of the stock prices over the last year
last_year_jnj = tickerDf_jnj.tail(252)['Close']
last_year_nvo = tickerDf_nvo.tail(252)['Close']
last_year_table = pd.DataFrame({'Johnson & Johnson': last_year_jnj, 'Novo Nordisk A/S': last_year_nvo})
last_year_table.columns = ['Last Year\'s Closing Price for ' + col + ' Stock' for col in last_year_table.columns]

# Print the last year's price table
print("\nLast Year's Closing Stock Prices for Johnson & Johnson and Novo Nordisk A/S Stocks")
print(last_year_table)

# Create a table of the percentage growth each year
yearly_growth_jnj = price_change_jnj.resample('Y').apply(lambda x: (1+x).prod() - 1) * 100
yearly_growth_nvo = price_change_nvo.resample('Y').apply(lambda x: (1+x).prod() - 1) * 100
yearly_growth_table = pd.concat([yearly_growth_jnj, yearly_growth_nvo], axis=1)
yearly_growth_table.columns = ['Johnson & Johnson', 'Novo Nordisk A/S']
yearly_growth_table.columns = ['Yearly Percentage Growth for ' + col + ' Stock' for col in yearly_growth_table.columns]

# Print the yearly growth table
print("\nYearly Percentage Growth for Johnson & Johnson and Novo Nordisk A/S Stocks")
print(yearly_growth_table.round(2))


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Yearly Percentage Growth for Johnson & Johnson and Novo Nordisk A/S Stocks
      Average Yearly Percentage Growth for Johnson & Johnson Stock   
2010                                           7.193584             \
2023                                           7.193584              

      Average Yearly Percentage Growth for Novo Nordisk A/S Stock  
2010                                          21.401067            
2023                                          21.401067            

Last Year's Closing Stock Prices for Johnson & Johnson and Novo Nordisk A/S Stocks
            Last Year's Closing Price for Johnson & Johnson Stock   
Date                                                                
2022-04-19                                         183.080002      \
2022-04-20                                         183.889999       
2022-04-21 